In [80]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline
import pprint as pp

import os
import numpy as np
import tensorflow as tf

import keras
from keras.preprocessing import image
from keras.applications.vgg19 import (
    VGG19, preprocess_input, decode_predictions)

In [81]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10090293610477475406
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11286223258
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2907253002775700172
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [82]:
# Load the VGG19 model
# https://keras.io/applications/#VGG19
model = VGG19(
    include_top=False, 
    weights='imagenet')
# model_original = VGG19(
#     include_top=True, 
#     weights='imagenet')

In [83]:
pp.pprint(model.layers)
pp.pprint(model.output)

<tf.Tensor 'block5_pool_2/MaxPool:0' shape=(?, ?, ?, 512) dtype=float32>


In [84]:
# Keras
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D
from keras.datasets import mnist

In [85]:
# add a global spatial average pooling layer
x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)


In [86]:
# this is the model we will train
model = Model(inputs=model.input, outputs=predictions)

In [87]:
# load image data
import glob
filtered_images_path= glob.glob('./dataset_1/filtered//imgs_with_filter/*')
unfiltered_imgs_path=glob.glob('./dataset_1/unfiltered//*')
images_path=filtered_images_path+unfiltered_imgs_path
images_list=[]
image_size = (224,224)
for img_path in images_path:
    image_stored=image.load_img(img_path,target_size=image_size)
    
    # Preprocess image for model prediction
# This step handles scaling and normalization for Xception
    x = image.img_to_array(image_stored)
#     x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    images_list.append(x)
    
# X=images_list[0]

In [88]:
X=np.array(images_list)

In [89]:
len(X)

2190

In [90]:
filtered_labels=np.full((len(filtered_images_path),2),(True,False),np.bool)
unfiltered_labels=np.full((len(unfiltered_imgs_path),2),(False,True),np.bool)
y=np.concatenate([filtered_labels,unfiltered_labels])
type(y)

numpy.ndarray

In [93]:
type(X_train)

numpy.ndarray

In [102]:
from numpy import random
def generator(features, labels, batch_size):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, 224,224, 3))
 batch_labels = np.zeros((batch_size,2))
 while True:
   for i in range(batch_size):
     # choose random index in features
     index= random.choice(len(features),1)
     batch_features[i] = (features[index])
     batch_labels[i] = labels[index]
   yield batch_features, batch_labels

In [103]:
#compile the model
for layer in model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
# model.fit_generator("")
model.fit(X_train,y_train,batch_size=10,epochs=1,shuffle=True)

## Image normalization and scaling
## selected layers trainable=true
## try with berkeley people data
## try with filter/nofilter on same image data
## data augmentation (rotate/mirror original image and filtered image)
## inception v1 


In [95]:
test_split=0.2

In [104]:
#splitting data into test and train
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_split, shuffle=True)


In [110]:
trainDatagen = image.ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False)

testDatagen = image.ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False)


trainDatagen.fit(X_train)
testDatagen.fit(X_test)

In [111]:
epochs=10
batch_size=10

steps_per_epoch=len(X_train)/batch_size
validation_steps=len(X_test)/batch_size

In [106]:
#Train the model
model.fit_generator(generator(X_train, y_train, batch_size), steps_per_epoch=steps_per_epoch, epochs=epochs , validation_data=generator(X_test,y_test,batch_size), validation_steps=validation_steps)

Epoch 1/10
176/175 [==============================] - 38s 215ms/step - loss: 3.1884 - acc: 0.4489 - val_loss: 2.9192 - val_acc: 0.4568
Epoch 2/10
176/175 [==============================] - 37s 212ms/step - loss: 2.9941 - acc: 0.4665 - val_loss: 3.2268 - val_acc: 0.4250
Epoch 3/10
176/175 [==============================] - 37s 212ms/step - loss: 3.1627 - acc: 0.4511 - val_loss: 2.8085 - val_acc: 0.4500
Epoch 4/10
176/175 [==============================] - 38s 214ms/step - loss: 3.1833 - acc: 0.4568 - val_loss: 2.9175 - val_acc: 0.4500
Epoch 5/10
176/175 [==============================] - 37s 213ms/step - loss: 3.1034 - acc: 0.4619 - val_loss: 3.0538 - val_acc: 0.4409
Epoch 6/10
176/175 [==============================] - 38s 214ms/step - loss: 3.0061 - acc: 0.4835 - val_loss: 2.9680 - val_acc: 0.4523
Epoch 7/10
176/175 [==============================] - 38s 214ms/step - loss: 2.9752 - acc: 0.4653 - val_loss: 3.3514 - val_acc: 0.4136
Epoch 8/10
155/175 [=========================>....] - E

KeyboardInterrupt: 

In [112]:
#Train the model
model.fit_generator(trainDatagen.flow(X_train, y_train, batch_size, shuffle=True), steps_per_epoch=steps_per_epoch, epochs=epochs , validation_data=testDatagen.flow(X_test,y_test,batch_size, shuffle=True), validation_steps=validation_steps)

Epoch 1/10
176/175 [==============================] - 36s 203ms/step - loss: 3.1460 - acc: 0.4523 - val_loss: 3.1223 - val_acc: 0.4315
Epoch 2/10
176/175 [==============================] - 35s 201ms/step - loss: 3.1436 - acc: 0.4545 - val_loss: 3.1223 - val_acc: 0.4315
Epoch 3/10
176/175 [==============================] - 35s 201ms/step - loss: 3.1434 - acc: 0.4545 - val_loss: 3.1223 - val_acc: 0.4315
Epoch 4/10
176/175 [==============================] - 36s 202ms/step - loss: 3.1434 - acc: 0.4545 - val_loss: 3.1223 - val_acc: 0.4315
Epoch 5/10
122/175 [===================>..........] - ETA: 8s - loss: 3.1478 - acc: 0.4541

KeyboardInterrupt: 

In [32]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)

ValueError: `steps_per_epoch=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps_per_epoch` or use the `keras.utils.Sequence` class.